In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#새 데이터 MF 클래스

In [3]:
# New MF class for training & testing
class NEW_MF():
    ###학습에 필요한 변수들 초기화작업###
    #ratings:평점. 사용자-아이템 평점 행렬이 들어옴
    #k:잠재요인(Latent Factor) 수 : p와q 행렬로 나뉘어질 때 각 행렬이 동일하게 가질 잠재요인 수
    #alpha:학습률 (Learning Rate), beta:정규화 개수, iterations:반복 횟수, verbose:중간출력옵션 - 따로 정해주는
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)

        # user_id, item_id를 R의 index와 매핑하기 위한 dictionary 생성 과정
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings): #enumerate(ratings)하면 - 0:첫번째 열이름 1:두번째열이름 반환됨
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)

        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T): #T 해서 사용자를index에서 column으로 만든 다음 위와 똑같이 진행함
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)

        # num_users : 사용자 수, num_items(아이템 수)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    # 현재의 P와 Q를 가지고 Root Mean Squared Error (RMSE) 계산
    def rmse(self):
        # 평점이 있는(0이 아닌) 요소의 인덱스
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            # 사용자 x, 아이템 y에 대해서 평점 예측치를 get_prediction()함수를 사용하여 계산
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            # 계산된 평점과 실제 값간의 오차를 errors리스트에 추가
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        # RMSE를 계산하여 반환
        return np.sqrt(np.mean(self.errors**2))

    #사용자i, 아이템j에 대해서 평점 예측하기
    def get_prediction(self, i, j):
        #전체 편향 + 사용자 i에 대한 편향 + 아이템 j에 대한 편향 + """잠재 요인 벡터 내적"""
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Stochastic gradient descent - 확률적 경사하강법 to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j) #예측
            e = (r - prediction) #e 오차 r 실제

            #개별 편향 업데이트
            #alpha : learning rate, 학습률. 반복마다 얼마나 큰 폭으로 업데이트할지에 대한 값
            #beta : regulation term, 정규화 계수. 편향이 과도하게 커지는 것을 방지하려 있음.
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

##### >>>>> (3)
    ### 분리된 test set을 넘겨받아 클래스 내부의 test_set을 만드는 함수 ###
    #.set_test() 부를때 넣어줄 테스트 데이터는 user_id, movid_Id, rating 세 열을 가진 데이터라
    #user_id, movie_id를 index로 바꿔준 값과 rating까지 가지는 데이터를 test_set으로 새로 만듦
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            # 현재 사용자, 아이템의 인덱스를 user_id_index에서 받아온다. (__init__에서 만듦)
            x = self.user_id_index[ratings_test.iloc[i, 0]]
            y = self.item_id_index[ratings_test.iloc[i, 1]]
            # 현재 사용자-아이템의 평점
            z = ratings_test.iloc[i, 2]
            test_set.append([x, y, z])

            # R을 사용해서 MF 모델을 학습하기 떄문에 test_set의 R을 제거
            self.R[x, y] = 0                    # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                         # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        # test set에 있는 각각의 (사용자, 아이템, 평점)에 대해서 RMSE를 구한다.
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        #사용자-k 행렬 P와 아이템-k 행렬 Q 정의
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        #편향 초기화. 학습이 진행되면서 각 사용자, 아이템마다의 편향값을 저장할 변수임.
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        #전체 평균 편향 : R에서 0이 아닌 모든 것들의 평균
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()       #train rsme
            rmse2 = self.test_rmse()  #test rsme
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 20 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])

    # Full user-movie rating matrix
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)

#추론

In [7]:
df_original = pd.read_csv("/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/10만행_데이터.csv", encoding='utf-8-sig', dtype = {"ISBN" : 'str'})
df = df_original.copy()

# 저장된 모델 파일을 로드
with open('/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/survey_ai_new_MF_model.md', 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
new_age = 29 #정수값
new_sex = 'F' #M or f
new_genre = '요리/살림' #genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

In [ ]:
#나이 성별 장르
pos = df[(df['age'] == new_age) & (df['sex'] == new_sex) & (df['fav_genre'] == new_genre)]

jump = -1
while pos.shape[0] == 0:
  new_age += jump
  pos = df[(df['age'] == new_age) & (df['sex'] == new_sex) & (df['fav_genre'] == new_genre)]

  #없으면 검색하는 나이 변경
  jump = jump - 1 if jump < 0 else jump + 1
  jump = -jump
  #검색 실패시 user_id랜덤에 break
  if abs(jump) > 100:
    pos = np.random.randint(1,900)
    break

#pos중 랜덤 행의 user_id
user_id_key = pos.sample(1)['user_id'].values[0]
print("user_id : ", user_id_key)

M = loaded_model.full_prediction()
M.shape

col = list(loaded_model.index_item_id.values())
val = M[loaded_model.user_id_index[user_id_key], : ]
res = pd.DataFrame(columns=col, data=val.reshape(1,-1))

res = res.T.sort_values(by=0, ascending=False)
res_index = res.index[0:10]
res_values = res.values[0:10]

print(res_index, res_values)

user_id :  676
Index([9788995235140, 9788971399019, 9788988996119, 9791186519905,
       9791192366395, 9788937400605, 9788937400537, 9788983920706,
       9788970415154, 9788988430323],
      dtype='int64') [[4.71274499]
 [4.67732602]
 [4.52041736]
 [4.34288293]
 [4.30698206]
 [4.29667011]
 [4.22338239]
 [4.1887588 ]
 [4.12918761]
 [4.12784714]]


#평가지표 만들기

In [38]:
# 저장된 모델 파일을 로드

"""with open('/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/survey_ai_new_MF_model.md', 'rb') as file:
    loaded_model = pickle.load(file)
answer_df = pd.read_csv("/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/사용자_정보_일관되게_만들기.csv", encoding='utf-8-sig')
#0.533625


with open('/content/drive/MyDrive/설문조사 AI 용 데이터/3. 새로운_모델_11_15_유저정보 일관 (과적합)/survey_ai_final_MF_model.md', 'rb') as file:
    loaded_model = pickle.load(file)
answer_df = pd.read_csv("/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/사용자_정보_일관되게_만들기.csv", encoding='utf-8-sig')
#0.5
"""

with open('/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/survey_ai_isbn_correction.md', 'rb') as file:
    loaded_model = pickle.load(file)
answer_df = pd.read_csv("/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/사용자_정보_일관되게_만들기_isbn_최신.csv", encoding='utf-8-sig')
#0.6256250000000001




In [39]:
def test(a, s, g):
  new_age = a #정수값
  new_sex = s #M or f
  new_genre = g #genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

  #나이 성별 장르
  pos = df[(df['age'] == new_age) & (df['sex'] == new_sex) & (df['fav_genre'] == new_genre)]

  jump = -1
  while pos.shape[0] == 0:
    new_age += jump
    pos = df[(df['age'] == new_age) & (df['sex'] == new_sex) & (df['fav_genre'] == new_genre)]

    #없으면 검색하는 나이 변경
    jump = jump - 1 if jump < 0 else jump + 1
    jump = -jump
    #검색 실패시 user_id랜덤에 break
    if abs(jump) > 100:
      pos = np.random.randint(1,900)
      break

  #pos중 랜덤 행의 user_id
  user_id_key = pos.sample(1)['user_id'].values[0]

  M = loaded_model.full_prediction()
  M.shape

  col = list(loaded_model.index_item_id.values())
  val = M[loaded_model.user_id_index[user_id_key], : ]
  res = pd.DataFrame(columns=col, data=val.reshape(1,-1))

  res = res.T.sort_values(by=0, ascending=False)
  res_index = res.index[0:10]
  res_values = res.values[0:10]

  return res_index, res_values


In [40]:
total_result = []

for i in range(10, 61):
  for j in ['M','F']:
    for k in ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']:
      dae = 4 if i>=50 else i//10

      #답안
      res_index, _ = test(i, j, k)
      #정답
      answer = answer_df[answer_df['분류'].str.contains(str(dae))
                  & answer_df['분류'].str.contains(j) & answer_df['장르'].str.contains(k)]
      answer = answer['ISBN 모음'].values
      answer = [a.strip() for a in eval(answer[0])]

      #정답 확인
      correct = 0
      for ind in res_index:
        if str(ind) in answer:
          correct = correct + 1

      #print(f"정답 : {correct/len(res_index): .2f}")
      total_result.append(correct/len(res_index))

np.array(total_result).mean()

0.71575

In [31]:
np.array(total_result)

array([1. , 1. , 1. , 1. , 0.8, 0.1, 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       0.9, 0.7, 0.1, 0.9, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.8, 0.2,
       1. , 1. , 1. , 1. , 0.8, 1. , 1. , 0.9, 1. , 0.1, 1. , 0.9, 1. ,
       1. , 1. , 1. , 1. , 1. , 0.7, 0.2, 1. , 1. , 1. , 1. , 1. , 1. ,
       0.7, 0.9, 1. , 0.2, 0.7, 1. , 1. , 1. , 1. , 1. , 0.9, 1. , 0.9,
       0.2, 1. , 1. , 1. , 0.9, 1. , 1. , 1. , 0.8, 1. , 0.3, 0.9, 0.9,
       0.8, 1. , 1. , 1. , 1. , 1. , 0.7, 0.1, 1. , 0.8, 1. , 0.9, 1. ,
       0.9, 0.9, 1. , 1. , 0.3, 0.8, 0.9, 1. , 0.9, 1. , 1. , 1. , 1. ,
       1. , 0.1, 1. , 1. , 1. , 1. , 1. , 1. , 0.7, 0.8, 1. , 0.1, 0.9,
       1. , 0.8, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

In [20]:
res_index, _ = test(15,'M','만화')
res_index

Index([9788988017197, 9788932012759, 9788974781040, 9788934902805,
       9788970996875, 9788995109113, 9788984310230, 9788953222984,
       9788988540022, 9788988017180],
      dtype='int64')

In [22]:
answer = answer_df[answer_df['분류'].str.contains(str(15//10)) & answer_df['분류'].str.contains('M') & answer_df['장르'].str.contains('만화')]
answer

,분류,장르,user_id 범위,ISBN 모음
2,10_M,만화,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","['9788973813438 ', '9788974781040 ', '97889747..."


In [26]:
answer = answer_df[answer_df['분류'].str.contains(str(15//10)) & answer_df['분류'].str.contains('M') & answer_df['장르'].str.contains('만화')]
answer

0.75

In [ ]:
print("내 정답 : ", res_index)
print("답안 : ", answer)


내 정답 :  Index([9788989351115, 9788932013916, 9788973372522, 9788934994985,
       9788982121487, 9788932312729, 9788986429343, 9788976266873,
       9788989549406, 9788955612837],
      dtype='int64')
답안 :  ['9788936303075', '9788970638331', '9788932008318', '9788982810831', '9788932312712', '9788971843383', '9788932000312', '9788988344064', '9788932008325', '9788973372911', '9788988344101', '9788972751021', '9788970631011', '9788936421915', '9788937418037', '9788937418044', '9788932004426', '9788936421854', '9788971841327', '9788970751535', '9788936420802', '9788973821297']
